从左向右的过程是神经网络的正向传播（也叫做前向传播，或者向前传播）。

假设我们有500条数据，20个特征，标签为3分类。我们现在要实现一个三层神经网络，这个神经网络的
架构如下：第一层有13个神经元，第二层有8个神经元，第三层是输出层。其中，第一层的激活函数是
relu，第二层是sigmoid。我们要如何实现它呢？

In [8]:
# 三层神经网络正向传播实现
import torch  # PyTorch深度学习框架核心库
import torch.nn as nn  # 包含神经网络层和损失函数
from torch.nn import functional as F  # 包含激活函数等操作

# 设置随机种子保证结果可复现
torch.random.manual_seed(420)  # 固定所有随机数生成器的种子

# 生成随机输入数据 (500个样本，每个样本20个特征)
X = torch.rand([500,20], dtype=torch.float32)  
# 批注：创建500×20的随机张量，数值范围[0,1)，float32是标准神经网络输入类型

# 生成随机标签数据 (500个样本，每个样本3个输出类别的one-hot编码)
y = torch.randint(low=0, high=3, size=[500,3], dtype=torch.int32)  
# 批注：创建500×3的随机整数张量，数值0/1/2，模拟3分类任务的one-hot标签
# 注意：实际one-hot应使用torch.eye(3)[y]转换，此处仅为示例

# 定义神经网络模型类
class Model(nn.Module):  # 继承nn.Module基类
    # 定义网络结构（初始化各层）
    def __init__(self, in_features, out_features):
        super(Model, self).__init__()  # 调用父类初始化方法
        self.linear1 = nn.Linear(in_features, 13)  # 第一全连接层：20输入→13输出
        self.linear2 = nn.Linear(13, 8)  # 第二全连接层：13输入→8输出
        self.output = nn.Linear(8, out_features)  # 输出层：8输入→3输出（对应3分类）
        # 批注：各层会自动初始化权重和偏置
        #super语法:super(当前类名, self).父类方法名(参数)
        #子类继承父类方法，只能继承除了init以外的方法，要想继承init，用super

    # 定义前向传播计算过程
    def forward(self, x):
        zhat1 = self.linear1(x)  # 第一层线性变换：x @ W1.T + b1
        sigma1 = torch.relu(zhat1)  # ReLU激活函数：max(0, zhat1)
        zhat2 = self.linear2(sigma1)  # 第二层线性变换
        sigma2 = torch.sigmoid(zhat2)  # Sigmoid激活：1/(1+e^(-zhat2))
        zhat3 = self.output(sigma2)  # 输出层线性变换（未归一化的logits）
        sigma3 = F.softmax(zhat3, dim=1)  # Softmax归一化：沿dim=1计算概率分布
        return sigma3  # 返回每个样本在3个类别上的概率（每行和为1）

# 实例化模型（重置随机种子保证初始化一致）
torch.random.manual_seed(420)  # 再次固定种子
net = Model(20, 3)  # 创建模型实例：输入特征20维，输出3分类
# 批注：此时会自动初始化各层的weight和bias参数

# 执行前向传播计算
output = net.forward(X)  # 等价于 net(X)
# 批注：输出形状为(500,3)，每行是样本在3个类别上的概率分布
# 例如：[[0.2,0.3,0.5], ...] 表示第一个样本属于3类的概率分别为20%/30%/50%
print (output)
print(net.linear1.weight.size())#w=[13,20],x=[500,20] ;w[13,20]*x[20,500]=[13,500]
print(net.linear2.weight.size())#w=[8,13],x=[13,500];w[8,13]*x[13,500]=[8,500]
print(net.output.weight.size())#w=[3,8],x=[8,500];w[3,8]*x[8,500]=[3,500]通过softmax变为[500,3]
print(net.linear1.bias.size())


tensor([[0.4140, 0.3496, 0.2365],
        [0.4210, 0.3454, 0.2336],
        [0.4011, 0.3635, 0.2355],
        ...,
        [0.4196, 0.3452, 0.2352],
        [0.4153, 0.3455, 0.2392],
        [0.4153, 0.3442, 0.2405]], grad_fn=<SoftmaxBackward0>)
torch.Size([13, 20])
torch.Size([8, 13])
torch.Size([3, 8])
torch.Size([13])


In [9]:
#
import torch
from torch.nn import MSELoss 
yhat=torch.randn(size=(50,),dtype=torch.float32 )
y=torch.randn(size=(50,),dtype=torch.float32)
criterion=MSELoss()
loss=criterion(yhat,y)
loss


tensor(1.9280)